In [1]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
# Extract text from inside

def preprocess(folder):
    no_of_paras = []
    current_dir = directory + "/"+folder
    base_cases = []
    noticed_cases = []
    paragraphs = []
    entailed_fragments = []
    list_of_files = sorted(os.listdir(current_dir))
    x = 1
    y = len(list_of_files)-x
    for i in range(y):
        print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/candidates"))
        a = f.read()
        base_cases.append(a)
        f.close()
       # g.close()
        
        arr = []
        no_of_paras.append(len(paragraph_filenames))

        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/candidates/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,no_of_paras,list_of_files,noticed_cases
    
train_base_cases,train_entailed_fragments, train_paragraphs,train_no_of_paras,train_list_of_files,train_noticed_cases = preprocess('task1_test')
    
#test_base_cases,test_entailed_fragments, test_paragraphs,test_no_of_paras = preprocess('task1_test')

Processing  0
Processing  1
Processing  2
Processing  3
Processing  4
Processing  5
Processing  6
Processing  7
Processing  8
Processing  9
Processing  10
Processing  11
Processing  12
Processing  13
Processing  14
Processing  15
Processing  16
Processing  17
Processing  18
Processing  19
Processing  20
Processing  21
Processing  22
Processing  23
Processing  24
Processing  25
Processing  26
Processing  27
Processing  28
Processing  29
Processing  30
Processing  31
Processing  32
Processing  33
Processing  34
Processing  35
Processing  36
Processing  37
Processing  38
Processing  39
Processing  40
Processing  41
Processing  42
Processing  43
Processing  44
Processing  45
Processing  46
Processing  47
Processing  48
Processing  49
Processing  50
Processing  51
Processing  52
Processing  53
Processing  54
Processing  55
Processing  56
Processing  57
Processing  58
Processing  59
Processing  60


In [ ]:
train_noticed_cases[0][:50]

In [6]:
data = train_base_cases + train_paragraphs

In [7]:
def gen_labels(data):
    labels = []
    for i in range(len(train_base_cases)):
        labels.append("base_case_"+train_list_of_files[i])
    for i in range(len(train_base_cases)):
        for j in range(200):
            labels.append("candidate_"+train_list_of_files[i]+"_"+str(j+1))
    return labels
labels = gen_labels(data)

In [8]:
import os
from os import listdir
from os.path import isfile, join
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
import gensim, logging
import time
import re
import nltk
import io
from nltk.corpus import stopwords
import numpy as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())

In [9]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=re.split('\W+',doc), tags=[self.labels_list[idx]])
iterator = DocIterator(data, labels)

In [11]:
model = gensim.models.Doc2Vec(vector_size=100, window=10, min_count=2, workers=4, alpha=0.025, min_alpha=0.001)
model.build_vocab(iterator)

print('done building vocabulary')
print('start training the model')
tic = time.time()
model.train(iterator,total_examples=model.corpus_count, epochs = 50)
toc = time.time()
print(toc-tic)
model.save("doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model")

2019-03-29 20:40:15,701 : INFO : collecting all words and their counts
2019-03-29 20:40:15,720 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-03-29 20:40:34,896 : INFO : PROGRESS: at example #10000, processed 25251214 words (1317076/s), 105251 word types, 10000 tags
2019-03-29 20:40:39,350 : INFO : collected 110939 word types and 12261 unique tags from a corpus of 12261 examples and 30874450 words
2019-03-29 20:40:39,351 : INFO : Loading a fresh vocabulary
2019-03-29 20:40:39,700 : INFO : effective_min_count=2 retains 93452 unique words (84% of original 110939, drops 17487)
2019-03-29 20:40:39,700 : INFO : effective_min_count=2 leaves 30856963 word corpus (99% of original 30874450, drops 17487)
2019-03-29 20:40:40,147 : INFO : deleting the raw counts dictionary of 110939 items
2019-03-29 20:40:40,149 : INFO : sample=0.001 downsamples 46 most-common words
2019-03-29 20:40:40,150 : INFO : downsampling leaves estimated 24623594 word corpus (79.8% of p

done building vocabulary
start training the model


2019-03-29 20:40:43,122 : INFO : EPOCH 1 - PROGRESS: at 1.48% examples, 470441 words/s, in_qsize 8, out_qsize 1
2019-03-29 20:40:44,132 : INFO : EPOCH 1 - PROGRESS: at 3.52% examples, 477067 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:40:45,142 : INFO : EPOCH 1 - PROGRESS: at 5.54% examples, 489913 words/s, in_qsize 8, out_qsize 0
2019-03-29 20:40:46,153 : INFO : EPOCH 1 - PROGRESS: at 7.89% examples, 501624 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:40:47,157 : INFO : EPOCH 1 - PROGRESS: at 10.10% examples, 505463 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:40:48,158 : INFO : EPOCH 1 - PROGRESS: at 12.07% examples, 510322 words/s, in_qsize 8, out_qsize 1
2019-03-29 20:40:49,169 : INFO : EPOCH 1 - PROGRESS: at 14.27% examples, 510310 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:40:50,179 : INFO : EPOCH 1 - PROGRESS: at 16.40% examples, 511152 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:40:51,186 : INFO : EPOCH 1 - PROGRESS: at 18.49% examples, 511099 words/s, in_qsize 8,

2019-03-29 20:41:52,797 : INFO : EPOCH 2 - PROGRESS: at 45.98% examples, 512303 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:41:53,798 : INFO : EPOCH 2 - PROGRESS: at 47.99% examples, 512690 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:41:54,814 : INFO : EPOCH 2 - PROGRESS: at 50.09% examples, 512794 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:41:55,832 : INFO : EPOCH 2 - PROGRESS: at 52.34% examples, 512617 words/s, in_qsize 8, out_qsize 0
2019-03-29 20:41:56,836 : INFO : EPOCH 2 - PROGRESS: at 54.47% examples, 512566 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:41:57,857 : INFO : EPOCH 2 - PROGRESS: at 56.49% examples, 511537 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:41:58,875 : INFO : EPOCH 2 - PROGRESS: at 58.82% examples, 511548 words/s, in_qsize 6, out_qsize 1
2019-03-29 20:41:59,875 : INFO : EPOCH 2 - PROGRESS: at 60.80% examples, 512234 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:42:00,881 : INFO : EPOCH 2 - PROGRESS: at 62.88% examples, 512647 words/s, in_qsiz

2019-03-29 20:43:02,973 : INFO : EPOCH 3 - PROGRESS: at 90.31% examples, 499442 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:43:03,974 : INFO : EPOCH 3 - PROGRESS: at 92.28% examples, 499437 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:43:04,977 : INFO : EPOCH 3 - PROGRESS: at 94.29% examples, 499424 words/s, in_qsize 8, out_qsize 1
2019-03-29 20:43:05,979 : INFO : EPOCH 3 - PROGRESS: at 96.36% examples, 499848 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:43:06,997 : INFO : EPOCH 3 - PROGRESS: at 98.43% examples, 500246 words/s, in_qsize 7, out_qsize 0
2019-03-29 20:43:07,668 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-29 20:43:07,677 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-29 20:43:07,678 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-29 20:43:07,679 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-29 20:43:07,679 : INFO : EPOCH - 3 : training on 30874450 raw w

...

2019-03-29 21:17:57,324 : INFO : EPOCH 47 - PROGRESS: at 95.21% examples, 527851 words/s, in_qsize 8, out_qsize 1
2019-03-29 21:17:58,327 : INFO : EPOCH 47 - PROGRESS: at 97.27% examples, 528089 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:17:59,346 : INFO : EPOCH 47 - PROGRESS: at 99.58% examples, 528596 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:17:59,494 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-29 21:17:59,497 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-29 21:17:59,502 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-29 21:17:59,503 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-29 21:17:59,504 : INFO : EPOCH - 47 : training on 30874450 raw words (24634055 effective words) took 46.6s, 529034 effective words/s
2019-03-29 21:18:00,534 : INFO : EPOCH 48 - PROGRESS: at 1.68% examples, 506473 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:18:01,545 : INFO : EPOCH 4

2019-03-29 21:19:02,242 : INFO : EPOCH 49 - PROGRESS: at 34.58% examples, 528950 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:19:03,251 : INFO : EPOCH 49 - PROGRESS: at 36.63% examples, 529540 words/s, in_qsize 8, out_qsize 0
2019-03-29 21:19:04,257 : INFO : EPOCH 49 - PROGRESS: at 38.82% examples, 529827 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:19:05,258 : INFO : EPOCH 49 - PROGRESS: at 40.93% examples, 529963 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:19:06,259 : INFO : EPOCH 49 - PROGRESS: at 43.05% examples, 530375 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:19:07,272 : INFO : EPOCH 49 - PROGRESS: at 45.37% examples, 530066 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:19:08,285 : INFO : EPOCH 49 - PROGRESS: at 47.52% examples, 530247 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:19:09,292 : INFO : EPOCH 49 - PROGRESS: at 49.58% examples, 530383 words/s, in_qsize 8, out_qsize 1
2019-03-29 21:19:10,293 : INFO : EPOCH 49 - PROGRESS: at 51.87% examples, 530250 words/s

2019-03-29 21:20:11,876 : INFO : EPOCH 50 - PROGRESS: at 69.82% examples, 459097 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:12,911 : INFO : EPOCH 50 - PROGRESS: at 72.06% examples, 460679 words/s, in_qsize 8, out_qsize 0
2019-03-29 21:20:13,919 : INFO : EPOCH 50 - PROGRESS: at 74.14% examples, 462719 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:14,927 : INFO : EPOCH 50 - PROGRESS: at 76.37% examples, 464569 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:15,944 : INFO : EPOCH 50 - PROGRESS: at 78.49% examples, 466136 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:16,957 : INFO : EPOCH 50 - PROGRESS: at 80.47% examples, 467399 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:17,962 : INFO : EPOCH 50 - PROGRESS: at 82.65% examples, 468665 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:18,980 : INFO : EPOCH 50 - PROGRESS: at 84.94% examples, 469933 words/s, in_qsize 7, out_qsize 0
2019-03-29 21:20:19,983 : INFO : EPOCH 50 - PROGRESS: at 87.15% examples, 471371 words/s

2384.7365453243256


2019-03-29 21:20:28,151 : INFO : saved doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model


In [8]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model")

2019-03-20 21:32:40,923 : INFO : loading Doc2Vec object from doc2vec_task_1.model
2019-03-20 21:32:42,263 : INFO : loading vocabulary recursively from doc2vec_task_1.model.vocabulary.* with mmap=None
2019-03-20 21:32:42,264 : INFO : loading trainables recursively from doc2vec_task_1.model.trainables.* with mmap=None
2019-03-20 21:32:42,264 : INFO : loading syn1neg from doc2vec_task_1.model.trainables.syn1neg.npy with mmap=None
2019-03-20 21:32:43,204 : INFO : loading wv recursively from doc2vec_task_1.model.wv.* with mmap=None
2019-03-20 21:32:43,206 : INFO : loading vectors from doc2vec_task_1.model.wv.vectors.npy with mmap=None
2019-03-20 21:32:44,009 : INFO : loading docvecs recursively from doc2vec_task_1.model.docvecs.* with mmap=None
2019-03-20 21:32:44,013 : INFO : loaded doc2vec_task_1.model


In [ ]:
model.docvecs.most_similar(labels[3],topn=500)

In [16]:
for i in range(61):
    #print("\nPredicting  For ",i)
    lst = []
    for j in range(200):
        #lst.append((model.docvecs.similarity(labels[i],labels[285+i*200+j]),j+1))
        lst.append((model.docvecs.similarity(labels[i],"candidate_"+train_list_of_files[i]+"_"+str(j+1)),j+1))
    lst.sort(reverse=True)
    cutoff = ((lst[0][0] + lst[1][0])/2)*0.90
    for j in range(10):
        if lst[j][0] > cutoff:
            print("t1-"+str(i+1),lst[j][1],"IITPd2v")

t1-1 48 IITPd2v
t1-2 154 IITPd2v
t1-3 180 IITPd2v
t1-3 112 IITPd2v
t1-4 74 IITPd2v
t1-4 176 IITPd2v
t1-4 112 IITPd2v
t1-4 182 IITPd2v
t1-4 153 IITPd2v
t1-4 20 IITPd2v
t1-4 193 IITPd2v
t1-4 137 IITPd2v
t1-4 65 IITPd2v
t1-4 124 IITPd2v
t1-5 163 IITPd2v
t1-5 130 IITPd2v
t1-5 172 IITPd2v
t1-5 190 IITPd2v
t1-5 174 IITPd2v
t1-5 186 IITPd2v
t1-5 23 IITPd2v
t1-5 5 IITPd2v
t1-6 62 IITPd2v
t1-6 85 IITPd2v
t1-6 168 IITPd2v
t1-7 86 IITPd2v
t1-7 157 IITPd2v
t1-7 7 IITPd2v
t1-7 178 IITPd2v
t1-7 22 IITPd2v
t1-7 147 IITPd2v
t1-8 197 IITPd2v
t1-8 88 IITPd2v
t1-8 157 IITPd2v
t1-9 71 IITPd2v
t1-10 187 IITPd2v
t1-10 180 IITPd2v
t1-10 164 IITPd2v
t1-11 111 IITPd2v
t1-11 49 IITPd2v
t1-11 105 IITPd2v
t1-11 195 IITPd2v
t1-11 2 IITPd2v
t1-12 50 IITPd2v
t1-12 56 IITPd2v
t1-12 8 IITPd2v
t1-12 138 IITPd2v
t1-12 7 IITPd2v
t1-12 83 IITPd2v
t1-12 52 IITPd2v
t1-12 106 IITPd2v
t1-12 192 IITPd2v
t1-12 131 IITPd2v
t1-13 95 IITPd2v
t1-13 160 IITPd2v
t1-13 75 IITPd2v
t1-14 156 IITPd2v
t1-14 102 IITPd2v
t1-15 18 IITPd2v
t1

In [18]:
correct_pred

431

In [11]:
total_true

1486

In [19]:
total_pred

855

In [30]:
325,1486,855

0.39569313593539707

In [31]:
588/2421

0.24287484510532836

In [35]:
468/1486

0.3149394347240915

In [39]:
import re
a = re.search("Baban","I am Baban Gain Baban GAin")
print(a)

<_sre.SRE_Match object; span=(5, 10), match='Baban'>
